# Title: SIFT Missense Predictions for Genomes and 1000 Genomes Data
# Course: MED263, "Bioinformatics Applications To Human Disease"
# Preparer: Nathaniel Delos Santos

# 1) Introduction
In this practical, you are going to use linux command line tools, the SIFT4G Variant annotator, and data from the 1000 Genomes Project to predict deleterious missense mutations from human samples. Predicting variant deleteriousness is an important part of analyzing human genome variants in disease, because it provides insight into which genes have been affected by a variant, and how bad the effect might be.  
After this tutorial, you should be able to:  
 - Download variant information from the Ensembl project
 - Download aligned sequence data from the 1000 Genomes project
 - Call variants from aligned sequence data
 - Annotate variants from aligned sequence data with deleteriousness and amino acid change predictions
 - Prioritize variants by a deleteriousness score
 - Perform data manipulation with basic command line tools in BASH

## 1.1) Download SIFT 4G
SIFT 4G, (Sorting Intolerant From Tolerant, For Genomes) uses variant calls to predict what amino acid substitutions occur, and how deleterious they are. SIFT 4G requires Java and a reference database to run. 

We will download SIFT 4G directly from their website at (http://sift.bii.a-star.edu.sg/sift4g/) using wget. Make sure wget is installed on your system.

In [ ]:
%%bash
wget -q http://sift.bii.a-star.edu.sg/sift4g/SIFT4G_Annotator_v2.4.jar

## 1.2) Download Homo Sapiens Database (GRCh38.78) for SIFTG
We must download the reference database for GRCh38, the newest version of the human genome reference available from Ensembl. We will download SIFT 4G's version of this database directly from their website. Make sure to choose GRCh38.78. Decompress the 

In [ ]:
%%bash
wget -q http://sift.bii.a-star.edu.sg/sift4g/public/Homo_sapiens/GRCh38.78.zip -O GRCh38.78.zip

In [ ]:
%%bash
unzip GRCh38.78.zip

## 1.3) SAMTools
SAMTools is a general toolkit for use with aligned sequencing data. We will use it here to call variants from sequence alignments, using the 'samtools mpileup' command. We will install version 1.4 here, since the specific version matters for our purposes.
Make sure that GCC and your build environment are up to date.

In [ ]:
%%bash
wget -q https://github.com/samtools/samtools/releases/download/1.4/samtools-1.4.tar.bz2 -O samtools-1.4.tar.bz2

In [ ]:
%%bash
tar -vxjf samtools-1.4.tar.bz2
cd samtools-1.4
./configure
make
cd ..

## 1.4) BCFTools
BCFTools is a general toolkit for use with variant call format (VCF) files. We will use it here to filter and query variants. We install version 1.4 here as we did for SAMTools

In [ ]:
%%bash
wget -q https://github.com/samtools/bcftools/releases/download/1.4/bcftools-1.4.tar.bz2 -O bcftools-1.4.tar.bz2

In [ ]:
%%bash
tar -vxjf bcftools-1.4.tar.bz2
cd bcftools-1.4
./configure
make
cd ..

# 2) Data

## 2.1) Craig Venter Germline Variations
Craig Venter's genome was among the first sequenced. These Variant Call Format (VCF) files summarize the variants observed in his genome from the GRCh38.78 reference.

In [ ]:
%%bash
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Venter.vcf.gz -O Venter.vcf.gz
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Venter.vcf.gz.tbi -O Venter.vcf.gz.tbi

## Question 1) 
How many variants are in the Venter VCF?

## Answer 1)

In [ ]:
%%bash
zcat Venter.vcf.gz|grep -v '#'|wc -l

3266109 Variants

## 2.2) James Watson Germline Variations
James Watson is famous for discovering the double helix structure of DNA with Francis Crick. He has his own tribute in VCF format here.

In [ ]:
%%bash
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Watson.vcf.gz -O Watson.vcf.gz
wget -q http://ftp.ensembl.org/pub/release-78/variation/vcf/homo_sapiens/Watson.vcf.gz.tbi -O Watson.vcf.gz.tbi

## 2.3) 1000 Genomes human sample exome data
The 1000 Genomes project was an international effort to catalog most variants with more than 1% frequency in the human population. It is a valuable source of human sequencing data. We will not be using the VCFs directly, but instead will be analyzing aligned sequences from a single human sample.

### 2.3.1) CRAM files
CRAM files are compressed sequence alignment files that use delta compression from a reference to store sequence information, rather than containining the sequence data themselves. Therefore, we must download the CRAM file, CRAM index, and the corresponding reference files to use them.

In [ ]:
%%bash
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/data/CEU/NA06984/exome_alignment/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram -O NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/data/CEU/NA06984/exome_alignment/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram.crai -O NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram.crai

The reference files for the CRAM file are downloaded below

In [ ]:
%%bash
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa -O GRCh38_full_analysis_set_plus_decoy_hla.fa
wget -q ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa.fai -O GRCh38_full_analysis_set_plus_decoy_hla.fa.fai

## Question 2)
From the README provided by the 1000 Genomes Project (ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/README.1000genomes.GRCh38DH.alignment) , what steps have already been performed for these CRAM files to make them ready for analysis?

## Answer 2)
1. Read alignment
2. Local realignment around Indels
3. Recalibration of base quality scores
4. Marking of duplicate reads
5. Merging multiple sequencing libraries into a single sample alignment file
6. Lossless compression using CRAM

# 3) Methods/Results
We will now run SIFT4G to predict the deleteriousness of variants found in the Venter VCF.

## 3.1) Analysis of Craig Venter germline variants
First we must decompress the gzipped VCF to an uncompressed VCF using zcat.

SIFT4G is run using java, so we must call it using 'java -jar', passing the SIFT4G program as the '-jar' option. The '-c' option will run SIFT4G in command line mode, and the '-t' option will cause SIFT4G to output additional annotations for each transcript of a gene affected. The '-i' option specifies the input VCF, in this case 'Venter.vcf'. The '-d' option specifies the database we will be using, in this case the GRCh38.78 database. The '-r' option will determine where the results of the SIFT annotation will be located relative to our current directory.

In [ ]:
%%bash
zcat Venter.vcf.gz > Venter.vcf
java -jar SIFT4G_Annotator_v2.4.jar -c -t -i Venter.vcf -d GRCh38.78  -r Venter.SIFT4G

## Question 3)
On Chromosome 17, how many variants are annotated? How many are unnannotated?

## Answer 3)
8831 annotated, 79089 unnannotated

### 3.1.1 SIFT 4G Output
The output of SIFT 4G includes a VCF file and an excel (.xls) file that describe the amino acid changes and the predicted deleteriousness of each variant. The excel file is formatted similarly to a tab-separated values file, with the exception of a carriage return ('\r') before each new line. We will use this to navigate the SIFT 4G output.

## Question 4)
How many columns Does the SIFT4G output contain? What does each column contain?

## Answer 4)

In [ ]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|head -n1|tr '\t' '\n'|cat -n

17 Columns, contents are described above.

## Question 5)
How many deleterious (not 'Low confidence') variants are found from these variants?

## Answer 5)

In [ ]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f1,2,3,4 \
|sort|uniq|wc -l

1561 deleterious variants.

## Question 6)
How many genes have deleterious variants? Output the list of genes names into a file. Display the gene names.

## Answer 6)

In [ ]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f7 \
|sort|uniq \
> Venter.SIFT4G.genes_with_deleterious_variants.txt
wc -l Venter.SIFT4G.genes_with_deleterious_variants.txt
cat Venter.SIFT4G.genes_with_deleterious_variants.txt

1186 genes. Gene names listed above.

### 3.1.2) SIFT Scores
SIFT scores less than 0.05 are considered deleterious. Anything greater is considered tolerated. Lower SIFT scores are considered more deleterious.

## Question 7)
What is the lowest SIFT score of the deleterious variants?

## Answer 7)

In [ ]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence' \
|cut -f1,2,3,4,13 \
|sort|uniq \
|sort -k1,1 -k2,2n \
|sort -k5,5n \
|head

0.0 is the lowest SIFT score.

## Question 8)
What variants are annotated with the lowest SIFT score? Output the chromosome, coordinate, reference base, alternate base, gene name, reference amino acid, alternate amino acid, amino acid position, and sift score into a file. Display the first 10 lines of this file.

## Answer 8)

In [ ]:
%%bash
cat Venter.SIFT4G/Venter_SIFTannotations.xls|cut -f1,2,3,4,7,10,11,12,13,17 \
|grep '^CHROM\|DELETERIOUS'|grep -v 'Low confidence' \
|awk '($9==0.0)||$1=="CHROM"' \
> Venter.SIFT4G.sift_score_0.txt
head -n10 Venter.SIFT4G.sift_score_0.txt

## 3.2) Analysis of James Watson germline variants

In [ ]:
%%bash
zcat Watson.vcf.gz > Watson.vcf
java -jar SIFT4G_Annotator_v2.4.jar -c -t -i Watson.vcf -d GRCh38.78  -r Watson.SIFT4G

## Question 9)
On Chromosome 17, how many variants are annotated? How many are unnannotated?

## Answer 9)
8476 annotated, 82316 unnannotated

## Question 10)
How many deleterious (not 'Low confidence') variants are found from these variants?

## Answer 10)

In [ ]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f1,2,3,4 \
|sort|uniq|wc -l

1970 deleterious variants.

## Question 11)
How many genes have deleterious variants? Output the list of genes names into a file. Display the gene names.

## Answer 11)

In [ ]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f7 \
|sort|uniq \
> Watson.SIFT4G.genes_with_deleterious_variants.txt
wc -l Watson.SIFT4G.genes_with_deleterious_variants.txt
cat Watson.SIFT4G.genes_with_deleterious_variants.txt

1528 genes. Gene names listed above.

## Question 12)
What genes do Craig Venter and James Watson both have deleterious variants in? How many genes is this?

## Answer 12)

In [ ]:
%%bash
join Venter.SIFT4G.genes_with_deleterious_variants.txt Watson.SIFT4G.genes_with_deleterious_variants.txt \
> Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt
wc -l Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt
cat Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt

Gene names provided above. 524 genes in common.

## Question 13)
What is the lowest SIFT score of the deleterious variants?

## Answer 13)

In [ ]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence' \
|cut -f1,2,3,4,13 \
|sort|uniq \
|sort -k1,1 -k2,2n \
|sort -k5,5n \
|head

0.0 is the lowest SIFT score.

## Question 14)
What variants are annotated with the lowest SIFT score? Output the chromosome, coordinate, reference base, alternate base, gene name, reference amino acid, alternate amino acid, amino acid position, and sift score into a file. Display the first 10 lines of this file.

## Answer 14)

In [ ]:
%%bash
cat Watson.SIFT4G/Watson_SIFTannotations.xls|cut -f1,2,3,4,7,10,11,12,13,17 \
|grep '^CHROM\|DELETERIOUS'|grep -v 'Low confidence' \
|awk '($9==0.0)||$1=="CHROM"' \
> Watson.SIFT4G.sift_score_0.txt
head -n10 Watson.SIFT4G.sift_score_0.txt

## 3.3) Analysis of 1000 Genomes Sample Human Data

### 3.3.1) Calling variants from aligned sequencing data
The 1000 Genomes exome sequencing data for this sample is not yet in VCF format. We must use samtools mpileup and bcftools call to convert it.

For samtools mpileup, we use the following options:
 - '-u' generate uncompressed VCF/BCF output. This saves time on compression and decompression, since we pipe to bcftools.
 - '-g' generate output in BCF format. This is a more compact binary format, ideal for transferring between programs.
 - '-f' the FASTA file used as reference for the CRAM file. Required to determine if something varies from the reference, and to decompress the CRAM data.
 
For bcftools call, we use the following options to call variants:
 - '-f GQ,GP' output genotype quality and genotype probability. We care about GQ for filtering.
 - '-v' output variant sites only. We don't care about sites that match the reference.
 - '-m' we use the multiallelic caller, upon recommendation by the samtools website.
 - '-O v' output VCF formatted file.
 - '-o' output variants to the specified file

We connect the output of samtools mpileup to the input of bcftools using a pipe '|'.

In [ ]:
%%bash
date
samtools-1.4/samtools mpileup \
-ugf GRCh38_full_analysis_set_plus_decoy_hla.fa \
NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.cram \
| bcftools-1.4/bcftools call \
-f GQ,GP \
-vmO v \
-o NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.gq.gp.vcf
date

### 3.3.2) Filtering variants by read depth, quality, and genotype quality
Not all variant calls are made equal. We want to avoid predicting the deleteriousness of variants that may not be real. So we use filtering to filter for the depth of sequencing at each variant coordinate, and the confidence the variant caller has in the variant. This is encapsulated in the DP, QUAL, and GQ fields.

The command bcftools filter is used to implement these filters.
 - '-i' specifies an expression for variants to include.
 - 'INFO/DP>10': We want raw read depth to be greater than 10
 - 'QUAL>20': We want the quality of any variant called here to be greater than 20
 - 'FMT/GQ>20': We want the genotype to be called with a confidence greater than 20.

We then combine these criteria using logical AND ('&&') to yield the final filter inclusion statement, '(QUAL>20)&&(INFO/DP>10)&&(FMT/GQ>20)'.

For more details on DP, QUAL, and GQ, see the guide from GATK (http://gatkforums.broadinstitute.org/gatk/discussion/1268/what-is-a-vcf-and-how-should-i-interpret-it).

In [ ]:
%%bash
date
bcftools-1.4/bcftools filter -i '(QUAL>20)&&(INFO/DP>10)&&(FMT/GQ>20)' \
-O v \
-o NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.vcf \
NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.gq.gp.vcf
date

## Question 15)
How many variants are in the VCF before filtering? How many after filtering?

## Answer 15)

In [ ]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.gq.gp.vcf|grep -v '^#'|wc -l
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.vcf|grep -v '^#'|wc -l

2254572 variants before filtering.
93617 variants after filtering.

### 3.3.3) Annotating variants with SIFT4G

In [ ]:
%%bash
java -jar SIFT4G_Annotator_v2.4.jar -c -t \
-i NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.vcf \
-d GRCh38.78  \
-r NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G

## Question 16)
On Chromosome 17, how many variants are annotated? How many are unnannotated?

## Answer 16)
1571 annotated, 2169 unnannotated

## Question 17)
How many deleterious (not 'Low confidence') variants are found from these variants?

## Answer 17)

In [ ]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f1,2,3,4 \
|sort|uniq|wc -l

1365 deleterious variants.

## Question 18)
How many genes have deleterious variants? Output the list of genes names into a file. Display the gene names.

## Answer 18)

In [ ]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence'|cut -f7 \
|sort|uniq \
> 1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
wc -l 1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
cat 1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt

1154 genes. Gene names listed above.

## Question 19)
What genes do Craig Venter, James Watson, and this 1000 Genomes sample All have deleterious variants in? How many genes is this?

## Answer 19)

In [ ]:
%%bash
join Venter_and_Watson.SIFT4G.genes_with_deleterious_variants.txt \
1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt \
> Venter_and_Watson_and_1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
wc -l Venter_and_Watson_and_1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt
cat Venter_and_Watson_and_1KGenomesSample.SIFT4G.genes_with_deleterious_variants.txt

Gene names provided above. 322 genes in common.

## Question 20)
What is the lowest SIFT score of the deleterious variants?

## Answer 20)

In [ ]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls|tail -n+2 \
|grep 'DELETERIOUS'|grep -v 'Low confidence' \
|cut -f1,2,3,4,13 \
|sort|uniq \
|sort -k1,1 -k2,2n \
|sort -k5,5n \
|head

0.0 is the lowest SIFT score.

## Question 21)
What variants are annotated with the lowest SIFT score? Output the chromosome, coordinate, reference base, alternate base, gene name, reference amino acid, alternate amino acid, amino acid position, and sift score into a file. Display the first 10 lines of this file.

## Answer 21)

In [ ]:
%%bash
cat NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20.SIFT4G\
/NA06984.alt_bwamem_GRCh38DH.20150826.CEU.exome.qual_gt_20.dp_gt_10.gq_gt_20_SIFTannotations.xls \
|cut -f1,2,3,4,7,10,11,12,13,17 \
|grep '^CHROM\|DELETERIOUS'|grep -v 'Low confidence' \
|awk '($9==0.0)||$1=="CHROM"' \
> 1KGenomesSample.SIFT4G.sift_score_0.txt
head -n10 1KGenomesSample.SIFT4G.sift_score_0.txt

# 4) References
1. Vaser R, Adusumalli S, Leng SN, Sikic M, Ng PC. SIFT missense predictions for genomes. Nat Protoc. 2016;11(1):1-9. (https://www.ncbi.nlm.nih.gov/pubmed/26633127)
2. Aken BL, Achuthan P, Akanni W, et al. Ensembl 2017. Nucleic Acids Res. 2017;45(D1):D635-D642. (https://www.ncbi.nlm.nih.gov/pubmed/27899575)
3. Auton A, Brooks LD, Durbin RM, et al. A global reference for human genetic variation. Nature. 2015;526(7571):68-74. (https://www.ncbi.nlm.nih.gov/pubmed/26432245)
4. Li H, Handsaker B, Wysoker A, et al. The Sequence Alignment/Map format and SAMtools. Bioinformatics. 2009;25(16):2078-9. (https://www.ncbi.nlm.nih.gov/pubmed/19505943)
5. Li H. A statistical framework for SNP calling, mutation discovery, association mapping and population genetical parameter estimation from sequencing data. Bioinformatics. 2011;27(21):2987-93. (https://www.ncbi.nlm.nih.gov/pubmed/21903627)
